In [106]:
from autogen_agentchat.agents import AssistantAgent
from  autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
import os
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain.agents import Tool

In [107]:
load_dotenv(override=True)
gemini_model  = "gemini-2.0-flash"
gemini_api_key = os.getenv("GEMINI_API_KEY")
gemini_client = OpenAIChatCompletionClient(model=gemini_model,api_key=gemini_api_key)

In [108]:
message = TextMessage(content="",source="user")
serper = GoogleSerperAPIWrapper()
lang_serper = Tool(name="web_search",func=serper.run,description="Used when need to search internet")
auto_serper = LangChainToolAdapter(lang_serper)
tools = [auto_serper]


In [109]:
lang_chain_file_management = FileManagementToolkit(root_dir="sandbox").get_tools()
for tool in lang_chain_file_management:
    tools.append(LangChainToolAdapter(tool))


In [110]:
for tool in tools:
    print(tool.name,tool.description)

web_search Used when need to search internet
copy_file Create a copy of a file in a specified location
file_delete Delete a file
file_search Recursively search for files in a subdirectory that match the regex pattern
move_file Move or rename a file from one location to another
read_file Read file from disk
write_file Write file to disk
list_directory List files and directories in a specified folder


In [111]:

prompt = """Your task is to find a one-way non-stop flight from JFK to LHR in nov 2025.
First search online for promising deals and cheapest.
Next, write all the deals to a file called flights.md with full details.
Finally, select the one you think is best and reply with a short summary.
Reply with the selected flight only, and only after you have written the details to the file."""



In [112]:
message = TextMessage(content=prompt,source="user")

In [113]:
autogen_agent = AssistantAgent(name="searcher",
                               model_client=gemini_client,
                               tools=tools,
                               reflect_on_tool_use=True)

In [114]:
from autogen_core import CancellationToken
from IPython.display import Markdown
response = await autogen_agent.on_messages(messages=[message],cancellation_token=CancellationToken())
Markdown(response.chat_message.content)

```tool_code
import datetime

deals = []

# November 2025 dates - assume prices from web search are valid for Nov 2025.
# Need to check closer to the time, but this gives an indication.
today = datetime.date.today()
year = 2025
month = 11

deals.append({
    "airline": "jetBlue",
    "price": 162,
    "date": datetime.date(year, month, 12),
    "departure_airport": "JFK",
    "arrival_airport": "LHR",
    "type": "one-way non-stop",
    "notes": "Wednesday departure, cheapest according to search"
})

deals.append({
    "airline": "jetBlue",
    "price": 163,
    "date": datetime.date(year, month, 17),
    "departure_airport": "JFK",
    "arrival_airport": "LHR",
    "type": "one-way non-stop",
    "notes": "Monday departure, could be slightly more expensive."
})

deals.append({
    "airline": "Various",
    "price": 174,
    "date": "November 2025",
    "departure_airport": "JFK",
    "arrival_airport": "LHR",
    "type": "one-way",
    "notes": "Cheapest found in web search. Specific airline/date not specified."
})

# Write to flights.md
with open("flights.md", "w") as f:
    f.write("# One-Way Non-Stop Flights from JFK to LHR - November 2025\n\n")
    for deal in deals:
        f.write(f"## {deal['airline']} - ${deal['price']}\n")
        f.write(f"* **Date:** {deal['date']}\n")
        f.write(f"* **Departure Airport:** {deal['departure_airport']}\n")
        f.write(f"* **Arrival Airport:** {deal['arrival_airport']}\n")
        f.write(f"* **Type:** {deal['type']}\n")
        f.write(f"* **Notes:** {deal['notes']}\n\n")

best_flight = deals[0] # Choose the first option (jetBlue $162 on Wed Nov 12th)
print(f"Selected Flight: {best_flight}")
```


In [115]:
message_two=TextMessage(content="OK proceed",source="user")
response = await autogen_agent.on_messages(messages=[message_two],cancellation_token=CancellationToken())
response.chat_message.content

'jetBlue, $162, November 12th, 2025, JFK to LHR, one-way non-stop. Wednesday departure.\n'

In [116]:
#team collebration agents
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import FunctionCallTermination
prompt = """Find a one-way non-stop flight from JFK to LHR in November 2025."""

primary_agent = AssistantAgent(
    name="searcher",
    model_client=gemini_client,
    tools=[auto_serper],
    reflect_on_tool_use=True,
    system_message="You are a helpful AI research assistant who looks for promising deals on flights. Incorporate any feedback you receive."

)

evaluation_agent = AssistantAgent(
    "evaluator",
    model_client=gemini_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' when your feedback is addressed."
)

In [117]:
text_termination = FunctionCallTermination("APPROVE")
team = RoundRobinGroupChat([primary_agent,evaluation_agent],termination_condition=text_termination,max_turns=10)

In [118]:
result = await team.run(task=prompt)
for message in result.messages:
    print(f'{message.source}"\n"{message.content}')

user"
"Find a one-way non-stop flight from JFK to LHR in November 2025.
searcher"
"[FunctionCall(id='function-call-2402173160610243608', arguments='{"query":"one-way non-stop flight from JFK to LHR in November 2025"}', name='web_search')]
searcher"
"[FunctionExecutionResult(content='Cheap Flights from New York (JFK) to London (LHR) start at $174 for one-way and $373 for round trip ... View $174 flight on Fri, Nov 28, 2025Opens in a new window. Need to get from New York to London? With fares from $457, we offer a great choice of food, drinks and onboard entertainment & WiFi. The cheapest round-trip flight from New York to London starts at $370 from Sat, Nov 8 to Mon, Nov 17. The cheapest one-way flight starts at $124 and departs on ... Kennedy International Airport (JFK) to Heathrow Airport (LHR), American Airlines offers flexible options that you can search according to schedule, budget, fare ... $162 per passenger.Departing Wed, Nov 12.One-way flight with jetBlue.Outbound direct fligh